<a href="https://colab.research.google.com/github/KhadkaBibek/prompt_engineering-/blob/main/prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pre.csv')
df

Mounted at /content/drive


,id,human,gpt,cleaned_human,cleaned_gpt
0,identity_0,"I""ve been feeling so sad and overwhelmed latel...","Hey there, I""m here to listen and support you....",feel sad overwhelm late work stress,hey listen support sound like work late tell w...
1,identity_0,"I recently got a promotion at work, which I th...",I can understand how it can be overwhelming wh...,recent got work thought would taken toll menta...,understand overwhelm face higher expect okay a...
2,identity_0,"Well, the workload has increased significantly...","It sounds like you""re dealing with a lot of pr...",well workload find hard maintain stay late fee...,sound like your deal lot perform succeed cruci...
3,identity_0,"I""ve been trying to prioritize my tasks and de...","It""s great to hear that you""re already impleme...",task also start break help stress feel like ma...,great hear your implement help progress take t...
4,identity_0,"You""re right. I haven""t really opened up about...","It""s completely normal to feel that way, but r...",your right open supervisor guess afraid appear...,normal feel way ask support strength weak star...
...,...,...,...,...,...
807080,identity_99469,"She""s been incredibly patient and understandin...",The fact that your girlfriend cares deeply abo...,she patient understand want support tell take ...,fact girlfriend care want work testament bond ...
807081,identity_99469,"I think it stems from unresolved trauma, Alex....","Trauma can often be a factor in addiction, Cha...",think stem trauma loss year ago turn way numb ...,trauma often factor addict connect loss addict...
807082,identity_99469,"It has affected my work, friendships, and over...","It""s completely understandable that addiction ...",affect work friendship feel like lost way addi...,understand addict setback aspect life setback ...
807083,identity_99469,I want to break free from the grip of addictio...,"Charlie, your determination to overcome addict...",want break free grip addict want regain contro...,addict rebuild life journey profession support...


In [2]:
!pip install groq
import os
from typing import Dict, List
from groq import Groq
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 2.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
os.environ['GROQ_API_KEY'] = "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"

# Models
DEFAULT_MODEL = 'llama-3.1-70b-versatile'


# CLIENTS TO INTERACT WITH GROQ
client = Groq()


In [6]:
# This function sends a series of messages (a conversation) to a chat API and retrieves the assistant's response.
#Sends a list of messages (formatted using the above functions) to a chatbot API
def chat_completion(
    message : List[Dict],
    model : str = DEFAULT_MODEL,
    temperature : float = 0.6,
    top_p: float = 0.9,

) -> str:
    try:
        response = client.chat.completions.create(
          messages = message,
          model = model,
          temperature = temperature,
          top_p = top_p,
    )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return "Error generating response."




In [7]:
## Zero-Shot Prompting
#A simplified wrapper for chat_completion for zero-shot learning.
def completion(
    prompt:str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,

) -> str:
    return chat_completion([user(prompt)],model = model,temperature = temperature,top_p = top_p,)
def user_and_print(prompt:str):
    print(f'+++++++++++++++{prompt}++++++++++++++++')
    response = completion(prompt, DEFAULT_MODEL)
    print(response, end = '\n\n')




In [ ]:
#These functions format messages for the chatbot API by assigning roles (user, assistant, or system) and associating them with the corresponding content.
## Message Formatting
def user(content: str):
    return {"role": "user", "content": content}

def assistant(content: str):
    return {"role": "assistant", "content": content}

def system(content: str):
    return {"role": "system", "content": content}

In [8]:
user_and_print(df['cleaned_human'][5])

+++++++++++++++thank understand remind face ill gather soon need find healthier++++++++++++++++
It sounds like you're trying to communicate a few things.  I'll try to break it down:

1. You want to remind someone about something.
2. You're not feeling well (you mentioned being "ill").
3. You need to gather something or someone soon.
4. You're looking for healthier options ( possibly for food or a lifestyle).

If I'm correct, here's a rephrased version of what you might be trying to say:

"Can you remind me about [something] soon? I'm not feeling well and I need to gather my thoughts. Also, I'm trying to find healthier options for [food/lifestyle], can you help me with that?"

Please feel free to correct me if I'm wrong or if you'd like me to rephrase anything.



In [9]:
# Generate few-short list
# Creates a few-shot prompt list by randomly sampling n examples from a dataset.
def random_sample(n:int, df:pd.DataFrame):
  few_shot = []
  random_data = df.sample(n)
  human_list = random_data['cleaned_human']
  gpt_list = random_data['cleaned_gpt']

  # Add system message
  few_shot.append(system("You are a chatbot. For each message, return a response based on user input."))

  # Add user and assistant examples
  for i,j in zip(human_list, gpt_list):
    few_shot.append(user(i))
    few_shot.append(assistant(j))
  return few_shot



In [17]:
# Generate response

def generate_response(few_shot:List[Dict], text:str):
  "Generate response from chat model"
  print(f"Type of few_shot: {type(few_shot)}")
  few_shot.append(user(text))
  response = chat_completion(few_shot)
  return response

In [11]:
# Evaluation Function
def evaluate_response(human_input:str, gpt_output :str, llama_response:str) -> Dict:

  """
 Evaluate the Llama response against the GPT response using semantic similarity.
 """
  model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
  embeddings = model.encode([gpt_output,llama_response])
  similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
  return {'human_input':human_input, 'gpt_output':gpt_output, 'llama_response':llama_response,'similarity_score':similarity_score  }



In [12]:
def few_shot_pipeline(df:pd.DataFrame, sample_size:int, test_cases : int ):
  """
  Run the few-shot prompt engineering pipeline and evaluate the response for multiple test cases
  """
  result = []
  few_shot_list = random_sample(sample_size, df)

  test_samples = df.sample(test_cases)
  for _, row in test_samples.iterrows():
    human_input = row['cleaned_human']
    gpt_output = row['cleaned_gpt']

    #Generate Llama response
    start_time = time.time()
    llama_response = generate_response(few_shot_list, human_input)
    elapsed_time = time.time() - start_time

    print(f'Llama response : {llama_response}')
    print(f'Response_time:{elapsed_time}')

    #Evaluate and store result
    evaluation_results = evaluate_response(human_input, gpt_output, llama_response)
    evaluation_results['response_time'] = elapsed_time
    result.append(evaluation_results)
    print(f"Evaluation Results: {evaluation_results}\n")
  return result



In [13]:
# Run pipeline with 5- shots promt and 3 test cases
result = few_shot_pipeline(df, 10,  2)
result


Type of few_shot: <class 'list'>
Llama response : safe space here judgment free allow express freely heard validated feel
Response_time:0.23874902725219727


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluation Results: {'human_input': 'think first step would pain open feel without judgment need safe space express heard', 'gpt_output': 'sound like compassion approach safe vital heal open feel member would process', 'llama_response': 'safe space here judgment free allow express freely heard validated feel', 'similarity_score': 0.46084604, 'response_time': 0.23874902725219727}

Type of few_shot: <class 'list'>
Llama response : understand face harsh words feel vulnerable need support safe space express feelings without fear judgment
Response_time:0.3112144470214844
Evaluation Results: {'human_input': 'feel overwhelm late face critic friend hard', 'gpt_output': 'therapist listen help sound like difficult tell conflict friend', 'llama_response': 'understand face harsh words feel vulnerable need support safe space express feelings without fear judgment', 'similarity_score': 0.4236665, 'response_time': 0.3112144470214844}



[{'human_input': 'think first step would pain open feel without judgment need safe space express heard',
  'gpt_output': 'sound like compassion approach safe vital heal open feel member would process',
  'llama_response': 'safe space here judgment free allow express freely heard validated feel',
  'similarity_score': 0.46084604,
  'response_time': 0.23874902725219727},
 {'human_input': 'feel overwhelm late face critic friend hard',
  'gpt_output': 'therapist listen help sound like difficult tell conflict friend',
  'llama_response': 'understand face harsh words feel vulnerable need support safe space express feelings without fear judgment',
  'similarity_score': 0.4236665,
  'response_time': 0.3112144470214844}]

In [14]:
# Generate few shot list with coT examples
def random_sample_cot(df:pd.DataFrame, n:int) -> List[Dict]:
  few_shot = []
  random_data = df.sample(n)
  human_list = random_data['cleaned_human']
  gpt_list = random_data['cleaned_gpt']

  # ADD system message
  few_shot.append(system("You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries."))

  # Add user and assistant examples.
  for human, gpt in zip(human_list, gpt_list):
    few_shot.append(user(human))
    few_shot.append(assistant(gpt))
  return few_shot


In [15]:
def few_shot_with_cot_pipeline(df:pd.DataFrame, sample_size:int, test_cases:int) -> List[Dict]:
    """
    Run the few-shot prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results = []
    few_shot_list = random_sample_cot(df, sample_size)
    print(few_shot_list)
    test_samples = df.sample(test_cases)
    for _,row in test_samples.iterrows():
      human_input = row['cleaned_human']
      gpt_output = row['cleaned_gpt']


      # Generate Llama response
      start_time = time.time()
      llama_response = generate_response(few_shot_list, human_input)
      elapsed_time = time.time() - start_time

    # Evaluate and store result
      evaluation_result = evaluate_response(human_input, gpt_output, llama_response)
      evaluation_result["response_time"] = elapsed_time
      results.append(evaluation_result)


    return results


In [16]:
result = few_shot_with_cot_pipeline(df, 5 ,2)
result

[{'role': 'system', 'content': 'You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries.'}, {'role': 'user', 'content': 'that tough question think would say loyal care person great humor love one see feel low'}, {'role': 'assistant', 'content': 'wonder hear let work healthier begin build shift away valid'}, {'role': 'user', 'content': 'well start week ago seem distant talk brush one day blue told love walk away'}, {'role': 'assistant', 'content': 'sound pain know breakup tough work feel right'}, {'role': 'user', 'content': 'well person growth gain better understand'}, {'role': 'assistant', 'content': 'that wonder hear path embark upon journey keep open mind may think'}, {'role': 'user', 'content': 'conflict seem relationship'}, {'role': 'assistant', 'content': 'understand trigger step find way think led misunderstand tension relationship'}, {'role': 'user', 'content': 'think set time day play guitar would would also give lack late'}, {'role': 'a

[{'human_input': 'dream write book one day idea share insight could other bring deep goal hope will put effort',
  'gpt_output': 'hear speak hope write book meaning endeavor allow share offer other step think take begin journey',
  'llama_response': 'beautiful goal inspiring share insight others dream book writing journey step take courage put words paper share world',
  'similarity_score': 0.6335074,
  'response_time': 0.2865328788757324},
 {'human_input': 'seen doctor none seem answer keep differ order endless test',
  'gpt_output': 'your seek answer feel like your make progress make feel clear answer',
  'llama_response': 'frustration seem endless cycle test, doctor, and uncertainty, no clear answer in sight.  think would take break, try approach differently, seek second opinion, or explore alternative options.',
  'similarity_score': 0.3161978,
  'response_time': 0.31807661056518555}]